<a href="https://colab.research.google.com/github/BigData-MachineLearning/P-Set3/blob/main/enet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Usamos la función pacman para llamar las bibliotecas necesarias para el análisis
install.packages('pacman')
library(pacman)
p_load(tidyverse,rio,tidymodels, glmnet)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
test <- import("/content/test_f.rds")
train <- import("/content/train_f.rds")

In [4]:
train["work_size"][is.na(train["work_size"])] <- 0
train["p5100"][is.na(train["p5100"])] <- 0

test["work_size"][is.na(test["work_size"])] <- 0
test["p5100"][is.na(test["p5100"])] <- 0


In [5]:
train <- train %>% select(p5000,p5010,p5090,p5100,p5100, nper, npersug,li,lp,mean_edad,jefe_mujer,educ_jefe,
                          tasa_afil,reg_salud,num_adult,num_minors,num_ocup,cost_arriendo,cuartos_per,
                          access_finan,bonificaciones,subs_alimeto,pay_alimento,pay_vivienda,pay_otros,primas,
                          mean_hrs_work,work_size,tasa_cotizantes,sub_empleo,disp_trabajar,dinero_trabajo,
                          dinero_arriendo, dinero_externo, dinero_remesas, ayuda_gob, ingtotugarr,lp)


In [6]:
train <- train |>
  mutate( ingtotugarr = as.numeric(ingtotugarr))


In [7]:
enet_spec <- linear_reg(penalty = tune(), mixture = tune()) |>
  set_mode("regression") |>
  set_engine("glmnet")

In [8]:
enet_recipe <-
  recipe(formula = ingtotugarr ~ ., data = train) %>%
  step_novel(all_nominal_predictors()) %>%
  step_dummy(all_nominal_predictors()) %>%
  step_zv(all_predictors()) %>%
  step_normalize(all_predictors())

In [9]:
enet_workflow <- workflow() %>%
  add_recipe(enet_recipe) %>%
  add_model(enet_spec)

In [10]:
tuning_grid <- grid_regular(penalty(range = c(-2, 2)), mixture(range=c(0,1)) ,levels = c(penalty = 50, mixture = 10))

In [11]:
enet_folds <- vfold_cv(train, v = 10)

In [13]:
enet_tune <- tune_grid(enet_workflow,
  resamples = enet_folds,
  grid = tuning_grid )

In [14]:
best_model <- select_best(enet_tune, metric = "rmse")

In [16]:
best_model

penalty,mixture,.config
<dbl>,<dbl>,<chr>
0.01,0.1111111,Preprocessor1_Model051


In [15]:
enet_final <- finalize_workflow(enet_workflow, best_model)

enet_final_fit <- fit(enet_final, data = train)

In [17]:
test<- augment(enet_final_fit, new_data = test)

In [18]:
submission<- test |> select(id,lp,.pred) |>
  rename(ingtotugarr=.pred) |>
  mutate(pobre = ifelse(ingtotugarr<lp,1,0)) |>
  select(id,pobre)
submission

id,pobre
<chr>,<dbl>
2a7ddc2779480d7f19834953,0
a0c2e751e582fd49d564f308,0
57273d19e8464a5ff66a582b,0
418d052ff7878940ab938601,0
212a37fc17016a3c78f76852,0
7b0b8c4814944383d6c8cef1,0
5e02916a18573d43c00da7c7,0
927ccc0298577d7009ef94ea,0
428494ae91e022e32c269115,0


In [19]:
rio::export(submission, "/content/regresion_elasticnet1.csv")